In [1]:
from docx import Document
from pathlib import Path
import os
import win32com.client as win32
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Cm
import csv
from PIL import Image
from io import BytesIO

#cargar los path y convertirlos
def cargar_path(path):
    convert_doc_to_docx(path)
    return list(path.glob('**\*.docx'))


#convertir doc en docx
def convert_doc_to_docx(path):
    # Initialize Word application
    word = win32.Dispatch("Word.Application")
    word.Visible = False  # Run in the background (headless)
    
    # check if docx already exists
    temp = list(path.glob('**/*.docx'))
    try:
        if not temp:
            temp = list(path.glob('**/*.doc'))
            for doc in temp:
                docx_path = doc.with_suffix('.docx')  # Change extension to .docx

                # Open the .doc file in Word
                doc_obj = word.Documents.Open(str(doc))

                # Save as .docx
                doc_obj.SaveAs(str(docx_path), FileFormat=16)  # 16 is the format code for .docx
                doc_obj.Close()  # Close the document
                # Delete the original .doc file
                doc.unlink()  # Using pathlib's unlink() method
    except Exception as e:
        print(f"Error converting {doc}: {e}")
    finally:
        word.Quit()



        
#extraer los datos de las tablas
def extract_patient_info(doc):
    data = {}
    mot={}
    #itero sobre las tablas del documento
    for index_table,table in enumerate(doc.tables):
        #tabla 1 es la que contiene estos datos
        if index_table==1:
            for row_index, row in enumerate(table.rows):
                if row_index>0:
                    for cell in row.cells:
                        if cell.text.lower().strip not in data:  
                            key=cell.text.split(':')[0].strip()
                            value=cell.text.split(':')[1].strip()
                            data[key]=value            
    return data


def extract_patient_measurements(path):
    #convierto el path en csv
    p=(str(path.with_suffix('.csv')).replace('Report.V3','Worksheet'))        
    data={}
    #genero un diccionario con el csv
    
    with open(p,mode='r') as file:
        csv_reader = csv.DictReader(file)
        
        #genero el limite de la tabla de mediciones
        limite=next((int(index) for index,row in enumerate(csv_reader) if 'Lucho Card' in row.values()),None)
        #seteo el iterador en 0
        file.seek(0)
        
        
        try:
            #extraigo los datos
            for index,row in enumerate(csv_reader):
                #genero el key con el primer valor de la row para el diccionario
                
                key=row.get('ï»¿Name','').strip().replace("'",'')
                if not key:
                    continue

                #chequeo si hay dos valores para cada medicion y si lo hay lo incorporo
                #los valores no numericos son unidades y los voy a tener que sacar manualmente despues
                if (
                    '2' in row 
                    and row['2'] is not None
                    and row['2']!='' 
                    and row['2'].replace('-','').replace('.','').isnumeric()

                ):
                    value=[row['1'],row['2']]

                else:
                    value=row['1']
                #agrego los datos bi dimensionales al diccionario
                data[key]=value
                #cheque
               # if 'Carotid'not in str(path)
                #lucho card marca el inicio de los calculo, asi que con ese tomo el index
                #si el indice es mayor al limite estamos en la segunda tabla. siempre sigue un patron donde el ante penultivo
                #elemento es el key que requiero y el valor esta en unir
                if limite is not None and index>limite+1:
                    #controlo el len de keys y v para modificar que variable sera key
                    if len(row.keys())==7:
                        k=-4
                        v=k+1
                    elif len(row.keys())==6:
                        k=-3
                        v=k+1
                    elif len(row.keys())==5:
                        k=-2
                        v=k+1
                    if row[list(row.keys())[k]] is not None:
                        key_2=row[list(row.keys())[k]].strip().replace("'",'')
                        value_2=row[list(row.keys())[v]]

                        #chequeo que valores 2 sea lista para no pner simbolos que compliquen la renderizacion
                        if isinstance(value_2,list):
                            data[key_2]=value_2[0]
                        else: data[key_2]=value_2
        except (IndexError, AttributeError) as e:
            # Log or handle specific exceptions if needed
            print(f"Error: {e}, no existe la tabla")    
        
        return data


def template_selector(path): 
    #por default elijo la plantilla de cardio
    template_path="E:\\plantillas\\cardiowest\\auto\\auto card.docx"
    tipo='card'
    path_str=str(path)
    #chqueo si en el path esta la palabra carotida si esta cambio a esa plantilla
    try:
        if 'Carotid' in path_str:
            template_path="E:\\plantillas\\cardiowest\\auto\\auto vc.docx"  
            tipo='carotid'
        elif 'Arteries' in path_str :#art
            template_path="E:\\plantillas\\cardiowest\\auto\\auto art.docx"  
            tipo='art'
        elif'Veins' in path_str:
            template_path="E:\\plantillas\\cardiowest\\auto\\auto ven.docx"   
            tipo='ven'
        else: 
            #el try es porque para distinguir cardio de stress neceisto acceder a una tabla, si la tabla no esta para el progrmaa
            #de esta manera con un try evito que se rompa si no esta y le asigo el valor default
            doc=Document(path)
            if doc.tables[2].rows[0].cells[0].text=='WMS':
                template=DocxTemplate("E:\\plantillas\\cardiowest\\auto\\auto stress.docx")
                tipo='stress'
    except (IndexError, AttributeError) as e:
        # Log or handle specific exceptions if needed
        print(f"Error: {e}, defaulting to 'card' template.")
        
    template = DocxTemplate(template_path)
    return template,tipo


def image_extractor(doc, template,tipo,image_width=Cm(8), image_height=Cm(5.36)):

    image_dict={}
    #extraer imagenes
    for rel in doc.part.rels:
        rel_obj = doc.part.rels[rel]
        if 'image' in rel_obj.reltype:
            image_data = rel_obj.target_part.blob
            image=Image.open(BytesIO(image_data))
            target= rel_obj.target_ref.split('.')[0].replace(r'media/','')
            #si el tipo de template es stres las primeras dos imagenes tiene un tamano distinto del resto
            
            if tipo=='stress' and int(target.replace('image','')) in [1,2]:
    
                if target.replace('image','')=='1':
                    image_dict[target]=InlineImage(template,
                                          BytesIO(image_data),
                                          width=Cm(16.23), height=Cm(8.22)
                                         )
                    
                else:
                        image_dict[target] = InlineImage(template,
                                          BytesIO(image_data),
                                          width=Cm(16.23), height=Cm(6.39)
                                         )
            else:          
                #default tamano de imagenes
                image_dict[target] = InlineImage(template,
                                                 BytesIO(image_data),
                                                 width=image_width,
                                                 height=image_height,

                                                )

    
    key=list(image_dict.keys())
    key = sorted(key, key=lambda image_name: int(image_name.replace('image', '')))
    image_dict={i: image_dict[i] for i in key}
    image_dict={'image':[{'key':k,'image':v} for k,v in image_dict.items()]}
    
    
    return image_dict


def mot_extractor(doc):
    
    table=doc.tables[2].rows[1].cells[0].tables[0]
    mot={}
    for index_r, row in enumerate(table.rows):
        #la fila 1 contiene la primera info,la ultima info en la 17 (apex)
        if 1 <= index_r <= 17:
            key = None
            values = []
            for index_c, cell in enumerate(row.cells):
                #las primeras cuatro celdas son segment ID
                #la 5 celda es el nombre del segmento
                #la 6 es baseline, 7 peak, 8 recovery 
                if index_c == 5:
                    key = cell.text.lower().replace(' ', '_')
                if 5 < index_c <= 8 and key:
                    values.append(cell.text)

            if key and values:  # Store the key-value pair only if both key and values exist
                mot[key] = values
                contex = {'mot': [{'key': k, 'motilidad': v} for k, v in mot.items()]}                                             
                                                                   
    return contex                                                              
        

def render_template(path):
    
    doc=Document(path)
    template,tipo=template_selector(path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam Date"]}.docx'
    print(save_path)
    print(tipo)
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        medidas= extract_patient_measurements(path)
        if 'stress' == tipo:
            mot=mot_extractor(doc)
        
    #calculo gradiente VDAD
        if 'TR Vmax' in medidas and isinstance(medidas['TR Vmax'],list):
            vel=(min([float(i) for i in medidas['TR Vmax']]))/100
            medidas['grad_vd_ad']=round(4*(vel**2),0)
            if len(medidas['TR Vmax'])>1:
                medidas['TR Vmax1']=round(float(medidas['TR Vmax'][0].replace('-','')),0)
                medidas['TR Vmax2']=round(float(medidas['TR Vmax'][1].replace('-','')),0)

          #genero 2 key de MV e para stress  
        if ('MV Vel E' in medidas 
            and isinstance(medidas['MV Vel E'],list) 
            and len(medidas['MV Vel E'])>1
           ):

            medidas['MV Vel E1']=round(float(medidas['MV Vel E'][0]),0)
            medidas['MV Vel E2']=round(float(medidas['MV Vel E'][1]),0)

        #calculo las E/e mediales//// faltan las laterales y el promedio varia segun el informe la info que puedo usar
        if ('Med Vel E' in medidas 
            and isinstance(medidas['Med Vel E'],list) 
            and len(medidas['Med Vel E'])>1
           ):

            medidas['Med Vel E1']=round(float(medidas['Med Vel E'][0].replace('-','')),0)
            medidas['Med Vel E2']=round(float(medidas['Med Vel E'][1].replace('-','')),0)
            medidas['e_e1']=round(medidas['MV Vel E1']/medidas['Med Vel E1'],0)
            medidas['e_e2']=round(medidas['MV Vel E2']/medidas['Med Vel E2'],0)

        #genero context
    if tipo=='stress':
        context={**info_pac,**medidas,'image': image['image'], 'mot': mot['mot']}
    elif tipo=='card':
        context={**info_pac,**medidas,'image': image['image']}
    else: 
        context={**info_pac,'image': image['image']}
        
    #quito los espacios en los context para que no haya conflicto
    context=(
        {key.replace(' ', '_').replace('/','_').replace('(','_').replace(')','_').
            replace('-','_'): value 
            for key, value in context.items()}
            )

    template.render(context)
    #template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')
        
        

In [ ]:
%%time
#creo la lista de los path a los docx
path=Path('C:\\Users\\Luciano\\Desktop\\test')
lista_de_trabajo=cargar_path(path)



In [ ]:
%%time
for index, i in enumerate(lista_de_trabajo):
    print(index)
    render_template(i)

In [ ]:
for index,i in enumerate(lista_de_trabajo):
    if index==0 or index==2:
        continue
    mot_extractor(i)


In [ ]:
doc=Document(lista_de_trabajo[3])
mot=mot_extractor(doc)
mot['mot']

In [ ]:
doc=Document(lista_de_trabajo[1])
template=DocxTemplate(lista_de_trabajo[0])
image=image_extractor(doc,template,tipo='stress')
print(image)
# Render the template with the context
template.render(image)
# Render the template with the context


# Save the rendered document
template.save('C:/Users/Luciano/Documents/temp/lala.docx')


In [ ]:
for rel in doc.part.rels:
    print(rel)
    rel_obj = doc.part.rels[rel]
    print(f"Relationship ID: {rel}")
    print(f"Relationship Type: {rel_obj.reltype}")
    print(f"Target Reference (target_ref): {rel_obj.target_ref}")

<!-- Render the first two images outside the table --> 
{{ image_dict['image1'] }}
{{ image_dict['image2'] }} 
<!-- Now, render the rest of the images in a table format --> 
{% set image_key = image_dict.keys()|list %}
{% for key in image_key[2:] %}
{{ image_dict[key] }}
{% endfor %}
